# CICIDS 2018 Full Dataset - Advanced IDS Training
## High-Accuracy Deep Learning Model for Intrusion Detection

This notebook trains a state-of-the-art intrusion detection model using the complete CICIDS 2018 dataset.

**Features:**
- Full CICIDS 2018 dataset (~16M samples)
- Advanced CNN + Attention architecture
- Comprehensive preprocessing and feature engineering
- Hyperparameter optimization
- Detailed evaluation metrics
- Model export for deployment

**Estimated Runtime:** 6-10 hours on Google Colab GPU

## 1. Setup and Installation

In [ ]:
# Install required packages - using compatible versions
!pip install -q tensorflow
!pip install -q scikit-learn
!pip install -q pandas numpy matplotlib seaborn
!pip install -q imbalanced-learn
!pip install -q boto3
!pip install -q tqdm

print("✓ All packages installed successfully!")

In [ ]:
# Import libraries
import os
import sys
import json
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.layers import (
    Dense, Dropout, BatchNormalization, Conv1D, MaxPooling1D,
    GlobalAveragePooling1D, LayerNormalization, Input,
    Concatenate, Reshape
)

# Try to import MultiHeadAttention, but don't fail if not available
try:
    from tensorflow.keras.layers import MultiHeadAttention
    ATTENTION_AVAILABLE = True
except ImportError:
    ATTENTION_AVAILABLE = False
    print("MultiHeadAttention not available in this TensorFlow version")

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    roc_curve, precision_recall_curve, f1_score, accuracy_score
)

# Imbalanced-learn
try:
    from imblearn.over_sampling import SMOTE
    SMOTE_AVAILABLE = True
except ImportError:
    SMOTE_AVAILABLE = False
    print("SMOTE not available - install imbalanced-learn if needed")

# AWS S3 for dataset download
import boto3
from botocore import UNSIGNED
from botocore.config import Config as BotoConfig

# Check GPU availability
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
if tf.config.list_physical_devices('GPU'):
    print("✓ GPU is available for training!")
else:
    print("⚠ No GPU found. Training will be slower on CPU.")
    print("Tip: In Colab, go to Runtime > Change runtime type > Hardware accelerator > GPU")

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("\n✓ All libraries imported successfully!")

## 2. Download CICIDS 2018 Dataset

In [ ]:
# Configuration
S3_BUCKET = 'cse-cic-ids2018'
S3_REGION = 'ca-central-1'
S3_PREFIX = 'Processed Traffic Data for ML Algorithms/'
DATA_DIR = Path('/content/cicids2018_data')
DATA_DIR.mkdir(exist_ok=True)

# Expected files in CICIDS 2018 dataset
EXPECTED_FILES = [
    'Friday-02-03-2018_TrafficForML_CICFlowMeter.csv',
    'Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv',
    'Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv',
    'Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv',
    'Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv',
    'Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv',
    'Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv',
    'Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv',
]

def download_from_s3(filename, local_path):
    """Download a file from S3"""
    try:
        s3_client = boto3.client(
            's3',
            region_name=S3_REGION,
            config=BotoConfig(signature_version=UNSIGNED)
        )
        
        s3_key = S3_PREFIX + filename
        print(f"Downloading {filename}...")
        s3_client.download_file(S3_BUCKET, s3_key, str(local_path))
        print(f"✓ Downloaded {filename}")
        return True
    except Exception as e:
        print(f"✗ Error downloading {filename}: {e}")
        return False

# List available files in S3
print("Discovering files in S3 bucket...")
try:
    s3_client = boto3.client(
        's3',
        region_name=S3_REGION,
        config=BotoConfig(signature_version=UNSIGNED)
    )
    
    available_files = []
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=S3_BUCKET, Prefix=S3_PREFIX)
    
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                key = obj['Key']
                if key.endswith('.csv') and 'TrafficForML' in key:
                    filename = os.path.basename(key)
                    size_mb = obj['Size'] / (1024 * 1024)
                    available_files.append((filename, size_mb))
    
    print(f"\nFound {len(available_files)} CSV files:")
    total_size = 0
    for filename, size in available_files:
        print(f"  - {filename} ({size:.2f} MB)")
        total_size += size
    print(f"\nTotal dataset size: {total_size:.2f} MB (~{total_size/1024:.2f} GB)")
    
    # Update expected files
    if available_files:
        EXPECTED_FILES = [f[0] for f in available_files]

except Exception as e:
    print(f"Warning: Could not list S3 files: {e}")
    print("Using default file list...")

print("\n" + "="*60)
print("Ready to download dataset")
print("="*60)

In [ ]:
# Download all CSV files
print("Starting download of CICIDS 2018 dataset...")
print("This may take 30-60 minutes depending on connection speed.\n")

downloaded_files = []
for filename in EXPECTED_FILES:
    local_path = DATA_DIR / filename
    
    # Skip if already downloaded
    if local_path.exists():
        print(f"✓ {filename} already exists, skipping...")
        downloaded_files.append(str(local_path))
        continue
    
    # Download file
    if download_from_s3(filename, local_path):
        downloaded_files.append(str(local_path))

print(f"\n✓ Downloaded {len(downloaded_files)}/{len(EXPECTED_FILES)} files successfully!")
print(f"Files saved to: {DATA_DIR}")

## 3. Data Loading and Exploration

In [ ]:
# Load and inspect the first file
if downloaded_files:
    sample_file = downloaded_files[0]
    print(f"Loading sample from: {sample_file}\n")
    
    # Read first few rows
    df_sample = pd.read_csv(sample_file, nrows=1000, low_memory=False)
    
    print(f"Dataset shape: {df_sample.shape}")
    print(f"\nColumn names ({len(df_sample.columns)} features):")
    for i, col in enumerate(df_sample.columns, 1):
        print(f"{i:3d}. {col}")
    
    print(f"\nLabel distribution in sample:")
    print(df_sample['Label'].value_counts())
    
    print(f"\nData types:")
    print(df_sample.dtypes.value_counts())
    
    print(f"\nMemory usage:")
    print(f"{df_sample.memory_usage(deep=True).sum() / 1024**2:.2f} MB for 1000 samples")
else:
    print("⚠ No files downloaded. Please run the download cell first.")

## 4. Data Preprocessing Pipeline

In [ ]:
class CICIDS2018Preprocessor:
    """Advanced preprocessing pipeline for CICIDS 2018 dataset"""
    
    def __init__(self):
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        self.feature_names = None
        self.protocol_map = {'TCP': 1, 'UDP': 2, 'ICMP': 3}
        
    def load_and_preprocess_file(self, filepath, sample_frac=None):
        """
        Load and preprocess a single CSV file
        
        Args:
            filepath: Path to CSV file
            sample_frac: Fraction of data to sample (None = use all)
        """
        print(f"\nProcessing: {os.path.basename(filepath)}")
        
        # Load data in chunks to handle memory efficiently
        chunks = []
        chunk_size = 100000
        
        for chunk in pd.read_csv(filepath, chunksize=chunk_size, low_memory=False):
            # Clean column names
            chunk.columns = chunk.columns.str.strip()
            chunks.append(chunk)
        
        df = pd.concat(chunks, ignore_index=True)
        print(f"  Loaded {len(df):,} samples")
        
        # Sample if requested
        if sample_frac and sample_frac < 1.0:
            df = df.sample(frac=sample_frac, random_state=42)
            print(f"  Sampled to {len(df):,} samples ({sample_frac*100}%)")
        
        # Clean data
        df = self._clean_data(df)
        
        return df
    
    def _clean_data(self, df):
        """Clean and prepare data"""
        # Remove duplicate rows
        initial_size = len(df)
        df = df.drop_duplicates()
        if len(df) < initial_size:
            print(f"  Removed {initial_size - len(df):,} duplicate rows")
        
        # Handle missing values
        df = df.replace([np.inf, -np.inf], np.nan)
        
        # Fill numeric columns with median
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())
        
        # Map protocol if exists
        if 'Protocol' in df.columns:
            df['Protocol'] = df['Protocol'].map(
                lambda x: self.protocol_map.get(str(x).upper(), 0)
            )
        
        # Create binary label: benign (0) vs malicious (1)
        if 'Label' in df.columns:
            df['Label'] = df['Label'].apply(
                lambda x: 0 if 'Benign' in str(x) else 1
            )
        
        return df
    
    def prepare_features(self, df):
        """
        Prepare features for training
        
        Returns:
            X: Feature array
            y: Label array
        """
        # Separate features and labels
        if 'Label' not in df.columns:
            raise ValueError("Label column not found")
        
        y = df['Label'].values
        
        # Drop non-feature columns
        feature_df = df.drop(columns=['Label'], errors='ignore')
        
        # Drop timestamp columns if present
        timestamp_cols = [col for col in feature_df.columns if 'Timestamp' in col]
        feature_df = feature_df.drop(columns=timestamp_cols, errors='ignore')
        
        # Store feature names
        self.feature_names = feature_df.columns.tolist()
        
        # Convert to numpy array
        X = feature_df.values
        
        print(f"  Features: {X.shape[1]}")
        print(f"  Samples: {X.shape[0]:,}")
        print(f"  Benign: {np.sum(y == 0):,} ({np.sum(y == 0) / len(y) * 100:.2f}%)")
        print(f"  Malicious: {np.sum(y == 1):,} ({np.sum(y == 1) / len(y) * 100:.2f}%)")
        
        return X, y
    
    def scale_features(self, X_train, X_val=None, X_test=None):
        """
        Scale features using StandardScaler
        """
        print("\nScaling features...")
        X_train_scaled = self.scaler.fit_transform(X_train)
        
        results = {'train': X_train_scaled}
        
        if X_val is not None:
            results['val'] = self.scaler.transform(X_val)
        
        if X_test is not None:
            results['test'] = self.scaler.transform(X_test)
        
        print("✓ Features scaled")
        return results

# Initialize preprocessor
preprocessor = CICIDS2018Preprocessor()
print("✓ Preprocessor initialized")

In [ ]:
# Process all downloaded files
print("="*60)
print("Processing CICIDS 2018 Dataset - 10% SAMPLE MODE")
print("="*60)
print("\nThis will process 10% of each file for faster training.")
print("Estimated time: 15-20 minutes\n")

# SAMPLE MODE ENABLED - Use 10% of data for faster training
USE_SAMPLE = True  # ✓ ENABLED - Change to False for full dataset
SAMPLE_FRAC = 0.1  # Using 10% of each file (~1.2-1.6M samples)

print(f"🚀 SAMPLE MODE: Using {SAMPLE_FRAC*100}% of data")
print(f"   Expected: ~1.2-1.6M samples")
print(f"   Training time: ~30-45 minutes\n")
print("💡 To use full dataset, set USE_SAMPLE = False\n")

all_data = []
for filepath in downloaded_files:
    try:
        df = preprocessor.load_and_preprocess_file(
            filepath, 
            sample_frac=SAMPLE_FRAC if USE_SAMPLE else None
        )
        all_data.append(df)
    except Exception as e:
        print(f"  ✗ Error processing {filepath}: {e}")

# Combine all data
print("\nCombining all files...")
combined_df = pd.concat(all_data, ignore_index=True)
print(f"✓ Total dataset size: {len(combined_df):,} samples")

# Shuffle data
print("\nShuffling dataset...")
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)
print("✓ Dataset shuffled")

# Display class distribution
print("\nFinal Class Distribution:")
label_counts = combined_df['Label'].value_counts()
print(f"  Benign (0): {label_counts.get(0, 0):,}")
print(f"  Malicious (1): {label_counts.get(1, 0):,}")

# Prepare features
print("\nPreparing features...")
X, y = preprocessor.prepare_features(combined_df)

# Free up memory
del combined_df, all_data
import gc
gc.collect()

print("\n" + "="*60)
print("Data Preprocessing Complete")
print("="*60)
print(f"Feature matrix shape: {X.shape}")
print(f"Label vector shape: {y.shape}")
print(f"\n✓ Ready for training with {len(X):,} samples!")

## 5. Train-Validation-Test Split

In [ ]:
# Split data: 70% train, 15% validation, 15% test
print("Splitting dataset...\n")

# First split: 70% train, 30% temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Second split: 50% of temp = 15% val, 15% test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print("Split sizes:")
print(f"  Training:   {len(X_train):,} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"  Validation: {len(X_val):,} samples ({len(X_val)/len(X)*100:.1f}%)")
print(f"  Test:       {len(X_test):,} samples ({len(X_test)/len(X)*100:.1f}%)")

print("\nClass distribution:")
print(f"  Train - Benign: {np.sum(y_train==0):,}, Malicious: {np.sum(y_train==1):,}")
print(f"  Val   - Benign: {np.sum(y_val==0):,}, Malicious: {np.sum(y_val==1):,}")
print(f"  Test  - Benign: {np.sum(y_test==0):,}, Malicious: {np.sum(y_test==1):,}")

# Free up memory
del X, y, X_temp, y_temp
gc.collect()

# Scale features
scaled = preprocessor.scale_features(X_train, X_val, X_test)
X_train_scaled = scaled['train']
X_val_scaled = scaled['val']
X_test_scaled = scaled['test']

print("\n✓ Data ready for training!")

## 6. Advanced Model Architecture

We'll implement a hybrid CNN + Attention architecture optimized for IDS:

In [ ]:
def create_advanced_ids_model(input_dim, model_type='cnn_attention'):
    """
    Create advanced IDS model
    
    Args:
        input_dim: Number of input features
        model_type: 'cnn_attention', 'deep_cnn', or 'dense'
    """
    inputs = Input(shape=(input_dim,))
    
    if model_type == 'cnn_attention':
        # Reshape for CNN
        x = Reshape((input_dim, 1))(inputs)
        
        # CNN blocks
        x = Conv1D(128, 3, activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling1D(2)(x)
        x = Dropout(0.3)(x)
        
        x = Conv1D(256, 3, activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling1D(2)(x)
        x = Dropout(0.3)(x)
        
        x = Conv1D(512, 3, activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)
        
        # Try to use MultiHeadAttention if available, otherwise skip
        try:
            attention_output = MultiHeadAttention(
                num_heads=8, key_dim=64, dropout=0.2
            )(x, x)
            x = LayerNormalization()(attention_output + x)
        except:
            # Fallback: just use the CNN output
            print("MultiHeadAttention not available, using CNN only")
        
        # Global pooling
        x = GlobalAveragePooling1D()(x)
        
        # Dense layers
        x = Dense(512, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        
        x = Dense(256, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.4)(x)
        
    elif model_type == 'deep_cnn':
        # Deep CNN architecture
        x = Reshape((input_dim, 1))(inputs)
        
        # Multiple CNN blocks
        for filters in [64, 128, 256, 512]:
            x = Conv1D(filters, 3, activation='relu', padding='same')(x)
            x = BatchNormalization()(x)
            x = Conv1D(filters, 3, activation='relu', padding='same')(x)
            x = BatchNormalization()(x)
            x = MaxPooling1D(2)(x)
            x = Dropout(0.3)(x)
        
        x = GlobalAveragePooling1D()(x)
        
        x = Dense(512, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        
        x = Dense(256, activation='relu')(x)
        x = Dropout(0.4)(x)
        
    else:  # dense
        # Deep dense network
        x = inputs
        
        for units in [1024, 512, 256, 128]:
            x = Dense(units, activation='relu')(x)
            x = BatchNormalization()(x)
            x = Dropout(0.4)(x)
    
    # Output layer
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model

# Create model
print("Creating advanced IDS model...\n")

# Choose model type - use 'deep_cnn' for better compatibility
MODEL_TYPE = 'deep_cnn'  # Options: 'cnn_attention', 'deep_cnn', 'dense'
print(f"Using model type: {MODEL_TYPE}")

model = create_advanced_ids_model(
    input_dim=X_train_scaled.shape[1],
    model_type=MODEL_TYPE
)

# Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)

# Display model summary
model.summary()

# Count parameters
total_params = model.count_params()
print(f"\nTotal parameters: {total_params:,}")
print(f"Model type: {MODEL_TYPE}")

## 7. Training Configuration and Callbacks

In [ ]:
# Setup training callbacks
checkpoint_dir = Path('/content/checkpoints')
checkpoint_dir.mkdir(exist_ok=True)

callbacks_list = [
    # Save best model
    callbacks.ModelCheckpoint(
        filepath=str(checkpoint_dir / 'best_model.h5'),
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    
    # Early stopping
    callbacks.EarlyStopping(
        monitor='val_auc',
        patience=15,
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce learning rate on plateau
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    
    # TensorBoard logging
    callbacks.TensorBoard(
        log_dir=str(checkpoint_dir / 'logs'),
        histogram_freq=1
    ),
    
    # CSV logger
    callbacks.CSVLogger(
        str(checkpoint_dir / 'training_log.csv'),
        append=True
    )
]

print("✓ Callbacks configured")
print("\nCallbacks:")
print("  - ModelCheckpoint: Save best model based on validation AUC")
print("  - EarlyStopping: Stop if no improvement for 15 epochs")
print("  - ReduceLROnPlateau: Reduce learning rate when stuck")
print("  - TensorBoard: Log metrics for visualization")
print("  - CSVLogger: Save training history to CSV")

## 8. Model Training

In [ ]:
# Training configuration - Optimized for 10% sample mode
BATCH_SIZE = 256
EPOCHS = 50  # Reduced for faster training with sample data

print("="*60)
print("Starting Model Training - 10% SAMPLE MODE")
print("="*60)
print(f"Model: {MODEL_TYPE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Max epochs: {EPOCHS}")
print(f"Training samples: {len(X_train_scaled):,}")
print(f"Validation samples: {len(X_val_scaled):,}")
print("\n⏱️ Estimated training time: 25-35 minutes")
print("Training will stop early if validation AUC stops improving.\n")

# Start training
start_time = datetime.now()
print(f"🚀 Training started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}\n")

history = model.fit(
    X_train_scaled, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val_scaled, y_val),
    callbacks=callbacks_list,
    verbose=1
)

end_time = datetime.now()
training_duration = end_time - start_time

print("\n" + "="*60)
print("Training Complete!")
print("="*60)
print(f"⏱️ Training duration: {training_duration}")
print(f"✓ Completed at: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📊 Actual epochs trained: {len(history.history['loss'])}")

## 9. Training History Visualization

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy
axes[0, 0].plot(history.history['accuracy'], label='Train')
axes[0, 0].plot(history.history['val_accuracy'], label='Validation')
axes[0, 0].set_title('Model Accuracy')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Loss
axes[0, 1].plot(history.history['loss'], label='Train')
axes[0, 1].plot(history.history['val_loss'], label='Validation')
axes[0, 1].set_title('Model Loss')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Precision
axes[1, 0].plot(history.history['precision'], label='Train')
axes[1, 0].plot(history.history['val_precision'], label='Validation')
axes[1, 0].set_title('Model Precision')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Recall
axes[1, 1].plot(history.history['recall'], label='Train')
axes[1, 1].plot(history.history['val_recall'], label='Validation')
axes[1, 1].set_title('Model Recall')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Recall')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig(checkpoint_dir / 'training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Training history plots saved")

## 10. Model Evaluation

In [ ]:
# Load best model
best_model_path = checkpoint_dir / 'best_model.h5'
if best_model_path.exists():
    print("Loading best model...")
    model = keras.models.load_model(str(best_model_path))
    print("✓ Best model loaded")

# Evaluate on test set
print("\nEvaluating on test set...")
test_results = model.evaluate(X_test_scaled, y_test, verbose=1)

print("\n" + "="*60)
print("Test Set Results")
print("="*60)
print(f"Loss:      {test_results[0]:.6f}")
print(f"Accuracy:  {test_results[1]:.4f} ({test_results[1]*100:.2f}%)")
print(f"Precision: {test_results[2]:.4f} ({test_results[2]*100:.2f}%)")
print(f"Recall:    {test_results[3]:.4f} ({test_results[3]*100:.2f}%)")
print(f"AUC:       {test_results[4]:.4f}")

# Generate predictions
print("\nGenerating predictions...")
y_pred_proba = model.predict(X_test_scaled, batch_size=BATCH_SIZE, verbose=1)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Additional metrics
f1 = f1_score(y_test, y_pred)
print(f"F1-Score:  {f1:.4f} ({f1*100:.2f}%)")
print("="*60)

In [ ]:
# Detailed classification report
print("\nClassification Report:")
print("="*60)
print(classification_report(
    y_test, y_pred,
    target_names=['Benign', 'Malicious'],
    digits=4
))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=['Benign', 'Malicious'],
    yticklabels=['Benign', 'Malicious']
)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(checkpoint_dir / 'confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Calculate metrics from confusion matrix
tn, fp, fn, tp = cm.ravel()
print("\nConfusion Matrix Breakdown:")
print(f"  True Negatives:  {tn:,}")
print(f"  False Positives: {fp:,}")
print(f"  False Negatives: {fn:,}")
print(f"  True Positives:  {tp:,}")

# False positive and false negative rates
fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
fnr = fn / (fn + tp) if (fn + tp) > 0 else 0
print(f"\nFalse Positive Rate: {fpr:.4f} ({fpr*100:.2f}%)")
print(f"False Negative Rate: {fnr:.4f} ({fnr*100:.2f}%)")

In [ ]:
# ROC curve
fpr_roc, tpr_roc, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr_roc, tpr_roc, color='darkorange', lw=2,
         label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.savefig(checkpoint_dir / 'roc_curve.png', dpi=150, bbox_inches='tight')
plt.show()

# Precision-Recall curve
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(recall_curve, precision_curve, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid(True, alpha=0.3)
plt.savefig(checkpoint_dir / 'precision_recall_curve.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ All evaluation plots saved")

## 11. Save Model and Artifacts

In [ ]:
# Save final model
output_dir = Path('/content/output')
output_dir.mkdir(exist_ok=True)

print("Saving model and artifacts...\n")

# Save model in different formats
model.save(str(output_dir / 'cicids2018_ids_model.h5'))
print("✓ Saved model in H5 format")

model.save(str(output_dir / 'cicids2018_ids_model_savedmodel'))
print("✓ Saved model in SavedModel format")

# Save preprocessor
with open(output_dir / 'preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)
print("✓ Saved preprocessor")

# Save training history
with open(output_dir / 'training_history.json', 'w') as f:
    json.dump({
        'history': {k: [float(v) for v in vals] for k, vals in history.history.items()},
        'training_duration': str(training_duration),
        'model_type': MODEL_TYPE,
        'total_params': int(total_params),
    }, f, indent=2)
print("✓ Saved training history")

# Save evaluation results
eval_results = {
    'test_metrics': {
        'accuracy': float(test_results[1]),
        'precision': float(test_results[2]),
        'recall': float(test_results[3]),
        'auc': float(test_results[4]),
        'f1_score': float(f1),
        'false_positive_rate': float(fpr),
        'false_negative_rate': float(fnr)
    },
    'confusion_matrix': cm.tolist(),
    'test_samples': int(len(y_test)),
    'model_type': MODEL_TYPE,
    'training_date': datetime.now().isoformat()
}

with open(output_dir / 'evaluation_results.json', 'w') as f:
    json.dump(eval_results, f, indent=2)
print("✓ Saved evaluation results")

# Save feature names
with open(output_dir / 'feature_names.json', 'w') as f:
    json.dump({'features': preprocessor.feature_names}, f, indent=2)
print("✓ Saved feature names")

print("\n" + "="*60)
print("All artifacts saved to:", output_dir)
print("="*60)

# List saved files
print("\nSaved files:")
for file in sorted(output_dir.iterdir()):
    size_mb = file.stat().st_size / (1024 * 1024) if file.is_file() else 0
    print(f"  - {file.name} ({size_mb:.2f} MB)")

## 12. Download Results to Local Machine

In [ ]:
# Create a zip file of all outputs for easy download
import shutil

print("Creating archive for download...\n")

archive_path = '/content/cicids2018_ids_model'
shutil.make_archive(archive_path, 'zip', output_dir)

archive_file = archive_path + '.zip'
archive_size = Path(archive_file).stat().st_size / (1024 * 1024)

print(f"✓ Archive created: {archive_file}")
print(f"  Size: {archive_size:.2f} MB")
print("\nDownload this file to use the trained model in your IDS system.")

# In Colab, you can download using:
try:
    from google.colab import files
    print("\nInitiating download...")
    files.download(archive_file)
except ImportError:
    print("\nNot running in Colab. Archive saved at:", archive_file)

## 13. Summary and Recommendations

In [ ]:
# Generate final summary
print("="*60)
print("TRAINING SUMMARY")
print("="*60)

print("\n📊 Dataset Statistics:")
print(f"  Total samples: {len(X_train_scaled) + len(X_val_scaled) + len(X_test_scaled):,}")
print(f"  Training samples: {len(X_train_scaled):,}")
print(f"  Validation samples: {len(X_val_scaled):,}")
print(f"  Test samples: {len(X_test_scaled):,}")
print(f"  Features: {X_train_scaled.shape[1]}")

print("\n🏗️ Model Architecture:")
print(f"  Type: {MODEL_TYPE}")
print(f"  Parameters: {total_params:,}")

print("\n⏱️ Training Time:")
print(f"  Duration: {training_duration}")
print(f"  Epochs completed: {len(history.history['loss'])}")

print("\n🎯 Performance Metrics:")
print(f"  Accuracy:  {test_results[1]*100:.2f}%")
print(f"  Precision: {test_results[2]*100:.2f}%")
print(f"  Recall:    {test_results[3]*100:.2f}%")
print(f"  F1-Score:  {f1*100:.2f}%")
print(f"  AUC-ROC:   {test_results[4]:.4f}")

print("\n🚨 Error Analysis:")
print(f"  False Positive Rate: {fpr*100:.2f}%")
print(f"  False Negative Rate: {fnr*100:.2f}%")

# Success criteria check
print("\n✅ Success Criteria:")
criteria = [
    ("Accuracy >= 95%", test_results[1] >= 0.95),
    ("Precision >= 90%", test_results[2] >= 0.90),
    ("Recall >= 90%", test_results[3] >= 0.90),
    ("F1-Score >= 90%", f1 >= 0.90),
    ("AUC >= 0.95", test_results[4] >= 0.95),
]

all_passed = True
for criterion, passed in criteria:
    status = "✓" if passed else "✗"
    print(f"  {status} {criterion}")
    if not passed:
        all_passed = False

if all_passed:
    print("\n🎉 All success criteria met!")
else:
    print("\n⚠️ Some criteria not met. Consider:")
    print("  - Training for more epochs")
    print("  - Adjusting hyperparameters")
    print("  - Trying different model architectures")
    print("  - Applying more advanced data augmentation")

print("\n📦 Output Files:")
print(f"  Location: {output_dir}")
print("  Files:")
print("    - cicids2018_ids_model.h5 (Keras model)")
print("    - preprocessor.pkl (Feature scaler)")
print("    - evaluation_results.json (Metrics)")
print("    - training_history.json (Training logs)")
print("    - feature_names.json (Feature list)")

print("\n🚀 Next Steps:")
print("  1. Download the model archive")
print("  2. Integrate into your IDS backend")
print("  3. Test with live traffic data")
print("  4. Monitor performance in production")
print("  5. Retrain periodically with new attack data")

print("\n" + "="*60)
print("Training Complete!")
print("="*60)